In [58]:
# import libraries
import sys
import os
os.environ['USE_PYGEOS'] = '0'

import pandas as pd
import geopandas as gpd

# import custom modules
sys.path.append('../../utils')
import data_paths
from gridding import GriddingEngine

from datetime import datetime

# Reload local modules on changes
%reload_ext autoreload
%autoreload 2

### Import Inventory files

In [ ]:
csee_nominal = gpd.read_file(data_paths.INVENTORY_PATH  + 'linesource_all_munich_2019_cold.gpkg')
csee_800m = gpd.read_file(data_paths.INVENTORY_PATH  + 'linesource_all_munich_2019_cold_800m.gpkg')
csee_2000m = gpd.read_file(data_paths.INVENTORY_PATH  + 'linesource_all_munich_2019_cold_2000m.gpkg')

In [83]:
for df in [csee_nominal, csee_800m, csee_2000m]:
    df['CO2_link_total'] = (df['PC_CO2(rep)'] + df['LCV_CO2(rep)'])*df.geometry.length/1000 #g/km
    df['CO_link_total'] = (df['PC_CO'] + df['LCV_CO'])*df.geometry.length/1000 #g/km
    df['NOx_link_total'] = (df['PC_NOx'] + df['LCV_NOx'])*df.geometry.length/1000 #g/km

print(f"Total CO2 emissions [kt] (800/1500/2000)m:\
      {csee_nominal['CO2_link_total'].sum()*1e-9:.2f}\
      {csee_800m['CO2_link_total'].sum()*1e-9:.2f}({(csee_800m['CO2_link_total'].sum()-csee_nominal['CO2_link_total'].sum()) / csee_nominal['CO2_link_total'].sum()*100:.2f}%)\
      {csee_2000m['CO2_link_total'].sum()*1e-9:.2f}({(csee_2000m['CO2_link_total'].sum()-csee_nominal['CO2_link_total'].sum()) / csee_nominal['CO2_link_total'].sum()*100:.2f}%)")


print(f"Total CO emissions [t] (800/1500/2000)m:\
      {csee_nominal['CO_link_total'].sum()/1e6:.2f}\
      {csee_800m['CO_link_total'].sum()/1e6:.2f}({(csee_800m['CO_link_total'].sum()-csee_nominal['CO_link_total'].sum()) / csee_nominal['CO_link_total'].sum()*100:.2f}%)\
      {csee_2000m['CO_link_total'].sum()/1e6:.2f}({(csee_2000m['CO_link_total'].sum()-csee_nominal['CO_link_total'].sum()) / csee_nominal['CO_link_total'].sum()*100:.2f}%)")

print(f"Total NOx emissions [t] (800/1500/2000)m:\
      {csee_nominal['NOx_link_total'].sum()/1e6:.2f}\
      {csee_800m['NOx_link_total'].sum()/1e6:.2f}({(csee_800m['NOx_link_total'].sum()-csee_nominal['NOx_link_total'].sum()) / csee_nominal['NOx_link_total'].sum()*100:.2f}%)\
      {csee_2000m['NOx_link_total'].sum()/1e6:.2f}({(csee_2000m['NOx_link_total'].sum()-csee_nominal['NOx_link_total'].sum()) / csee_nominal['NOx_link_total'].sum()*100:.2f}%)")



Total CO2 emissions [kt] (800/1500/2000)m:      22.46      22.33(-0.61%)      22.67(0.91%)
Total CO emissions [t] (800/1500/2000)m:      1549.23      1539.75(-0.61%)      1563.39(0.91%)
Total NOx emissions [t] (800/1500/2000)m:      64.16      63.77(-0.61%)      64.75(0.92%)


In [ ]:
# compare cold starts spatially by road class
links_nominal = gpd.read_file(data_paths.VISUM_FOLDER_PATH + 'visum_links.gpkg')
links_800m = gpd.read_file(data_paths.VISUM_FOLDER_PATH + 'visum_links_800m.gpkg')
links_2000m = gpd.read_file(data_paths.VISUM_FOLDER_PATH + 'visum_links_2000m.gpkg')

links_nominal = links_nominal.groupby('road_type')[['PC_cold_starts', 'LCV_cold_starts']].sum().sum(axis = 1)
links_nominal.name = 'nominal'
links_800m = links_800m.groupby('road_type')[['PC_cold_starts', 'LCV_cold_starts']].sum().sum(axis = 1)
links_800m.name = '800m'
links_2000m = links_2000m.groupby('road_type')[['PC_cold_starts', 'LCV_cold_starts']].sum().sum(axis = 1)
links_2000m.name = '2000m'

df_road_type = pd.concat([links_nominal, links_800m, links_2000m], axis=1)

In [80]:
df_road_type['diff_800m'] = (df_road_type['800m'] - df_road_type['nominal'])/df_road_type['nominal']*100
df_road_type['diff_2000m'] = (df_road_type['2000m'] - df_road_type['nominal'])/df_road_type['nominal']*100

df_road_type

,nominal,800m,2000m,diff_800m,diff_2000m
road_type,,,,,
Access-residential,8.067932e+05,8.273932e+05,7.968336e+05,2.553315,-1.234475
Distributor/Secondary,2.127309e+06,2.106709e+06,2.137269e+06,-0.968359,0.468181
Motorway-Nat,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN
TrunkRoad/Primary-City,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN
TrunkRoad/Primary-National,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN


In [93]:
# spatial plots of cold start emissions
# perform gridding
# initialize gridding object

tno_grid = gpd.read_file(data_paths.TNO_1km_GRID)

gridding_obj = GriddingEngine(input_grid=tno_grid, crs=csee_nominal.crs) # hot and cold inventory have the same crs

em_cold_columns = [f'{v}_{c}' for v in ['PC', 'LCV'] for c in ['CO2(rep)', 'CO', 'NOx']]
out_grid_nominal = gridding_obj.overlay_grid(csee_nominal, value_columns= em_cold_columns,
                                          source_type='line_kilometer')

out_grid_nominal['CO2(rep)_nominal'] = out_grid_nominal['PC_CO2(rep)'] + out_grid_nominal['LCV_CO2(rep)']
out_grid_nominal['CO_nominial'] = out_grid_nominal['PC_CO'] + out_grid_nominal['LCV_CO']
out_grid_nominal['NOx_nominial'] = out_grid_nominal['PC_NOx'] + out_grid_nominal['LCV_NOx']

out_grid_800m = gridding_obj.overlay_grid(csee_800m, value_columns= em_cold_columns,
                                          source_type='line_kilometer')

out_grid_800m['CO2(rep)_800m'] = out_grid_800m['PC_CO2(rep)'] + out_grid_800m['LCV_CO2(rep)']
out_grid_800m['CO_800m'] = out_grid_800m['PC_CO'] + out_grid_800m['LCV_CO']
out_grid_800m['NOx_800m'] = out_grid_800m['PC_NOx'] + out_grid_800m['LCV_NOx']

out_grid_2000m = gridding_obj.overlay_grid(csee_2000m, value_columns= em_cold_columns,
                                          source_type='line_kilometer')

out_grid_2000m['CO2(rep)_2000m'] = out_grid_2000m['PC_CO2(rep)'] + out_grid_2000m['LCV_CO2(rep)']
out_grid_2000m['CO_2000m'] = out_grid_2000m['PC_CO'] + out_grid_2000m['LCV_CO']
out_grid_2000m['NOx_2000m'] = out_grid_2000m['PC_NOx'] + out_grid_2000m['LCV_NOx']

In [94]:
spatial_comparison = pd.concat([out_grid_nominal[['CO2(rep)_nominal', 'CO_nominial', 'NOx_nominial']],
                                out_grid_800m[['CO2(rep)_800m', 'CO_800m', 'NOx_800m']],
                                out_grid_2000m[['CO2(rep)_2000m', 'CO_2000m', 'NOx_2000m', 'geometry']]], axis=1)

spatial_comparison = gpd.GeoDataFrame(spatial_comparison, geometry='geometry', crs=out_grid_nominal.crs)

In [95]:
spatial_comparison['diff_800m'] = (spatial_comparison['CO2(rep)_800m'] - spatial_comparison['CO2(rep)_nominal'])/spatial_comparison['CO2(rep)_nominal']*100
spatial_comparison['diff_2000m'] = (spatial_comparison['CO2(rep)_2000m'] - spatial_comparison['CO2(rep)_nominal'])/spatial_comparison['CO2(rep)_nominal']*100

In [96]:
spatial_comparison.to_file(data_paths.INVENTORY_PATH  + 'spatial_comparison_cold_start_buffer_distance.gpkg')

In [97]:
spatial_comparison

,CO2(rep)_nominal,CO_nominial,NOx_nominial,CO2(rep)_800m,CO_800m,NOx_800m,CO2(rep)_2000m,CO_2000m,NOx_2000m,geometry,diff_800m,diff_2000m
0,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,"POLYGON ((674785.339 5335643.053, 676024.875 5...",NaN,NaN
1,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,"POLYGON ((674757.026 5336569.208, 675996.361 5...",NaN,NaN
2,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,"POLYGON ((676053.386 5334754.926, 677293.121 5...",NaN,NaN
3,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,"POLYGON ((676024.875 5335681.079, 677264.409 5...",NaN,NaN
4,7.758155e+06,5.403839e+05,22532.396728,3.335830e+06,2.317426e+05,9645.666347,6.683711e+06,4.657564e+05,19426.649043,"POLYGON ((675996.361 5336607.233, 677235.694 5...",-57.002281,-13.849218
...,...,...,...,...,...,...,...,...,...,...,...,...
321,7.756723e+06,5.365430e+05,22266.136889,7.305367e+06,5.061738e+05,21030.157965,8.219582e+06,5.690639e+05,23630.135821,"POLYGON ((700912.815 5333719.72, 702152.917 53...",-5.818896,5.967204
322,9.479692e+07,6.558125e+06,272182.821462,1.152841e+08,7.989880e+06,332017.461802,9.232298e+07,6.383212e+06,264815.907522,"POLYGON ((700880.292 5334645.845, 702120.193 5...",21.611621,-2.609729
323,2.792313e+07,1.930203e+06,80065.475186,2.336261e+07,1.609880e+06,66633.407914,2.455612e+07,1.699350e+06,70543.777558,"POLYGON ((700847.765 5335571.971, 702087.465 5...",-16.332421,-12.058159
324,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,"POLYGON ((702152.917 5333763.401, 703393.017 5...",NaN,NaN
